In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# SBERT 모델 로드
embedder = SentenceTransformer("jhgan/ko-sbert-sts")

# 데이터 준비
review = pd.read_csv('review_with_nouns_adjectives.csv')
review['리뷰 텍스트'] = review['리뷰 텍스트'].fillna('').astype(str)
texts = review['리뷰 텍스트'].tolist()

# 배치 처리로 임베딩 생성
batch_size = 32
embeddings = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    batch_embeddings = embedder.encode(batch_texts, convert_to_tensor=True)
    embeddings.append(batch_embeddings.cpu().numpy())

# 배치를 결합하여 하나의 배열로 생성
embeddings_np = np.vstack(embeddings)

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=1.0, min_samples=5, metric='cosine')
labels = dbscan.fit_predict(embeddings_np)

# 결과 저장
review['cluster'] = labels
print(review[['리뷰 텍스트', 'cluster']].head())

: 

In [47]:
from sklearn.cluster import DBSCAN
import numpy as np


# Tensor를 Numpy 배열로 변환
embeddings_np = embeddings.cpu().numpy()

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=1.0, min_samples=5, metric='cosine')
labels = dbscan.fit_predict(embeddings_np)

# 클러스터 레이블을 데이터프레임에 추가
review['cluster'] = labels

# 클러스터링 결과 확인
print(review[['리뷰 텍스트', 'cluster']].head())

: 